etri api를 사용하여 형태소 분석 후 명사만 가져와 토큰화한다.  
* 형태소 분석 (문어/구어) : "morp"

# import library

In [1]:
import urllib3
import json
import pandas as pd

pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings("ignore")

# upload book data

In [2]:
df1 = pd.read_json('/home/user_1/medistream-recsys/Script/YS/df_book_clean.json')
df = df1.copy()

In [3]:
# api_key = '8e216d4f-bfae-4cc4-ae6f-25f2dc4968c5' # 혜빈 키
api_key = 'de384842-9f74-42c9-a0e5-036ecd76eab5' # 영수님 키

언어 분석 기술 문어/구어 중 한가지만 선택해 사용

In [7]:
# // 언어 분석 기술(문어)
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 

# preprocessing

### 1. api로 불러온 data에서 필요한 정보 추출

In [8]:
class ETRISentenceAnalysis:
    def __init__(self, api_key: dict, url: str):
        self.api_key = api_key
        self.url = url

        self.http = urllib3.PoolManager()
  
    def _make_request_json(self, text: str, analysis_code: int) -> dict:
        return {
          "access_key": self.api_key,
          "argument": {
              "text": text,
              "analysis_code": analysis_code
              }
          }

    def _request(self, request_json: dict) -> object:
        return self.http.request(
            "POST",
            openApiURL,
            headers={"Content-Type": "application/json; charset=UTF-8"},
            body=json.dumps(request_json)
        )

    # 형태소 분석 결과 출력
    def get_analyzed_sentence(self, sentence: str, analysis_code: int) -> dict:
        def _get_return_object(sentence: str) -> dict:
            data = str(response.data, 'utf-8')
            data = json.loads(data)


            return data.get('return_object') 

        request_json = self._make_request_json(sentence, analysis_code)
        response = self._request(request_json)

        assert response.status == 200, f'Error {response.status}'

        return _get_return_object(response)

In [9]:
etri_sentence_analysis = ETRISentenceAnalysis(api_key, openApiURL)

### 2. description 없는 내용 채우기
description에 내용이 없을 경우 형태소 분석 결과가 'none'으로 나와 error가 발생한다.  
값이 없는 row들이 많지 않아 각 도서의 목차를 검색하여 값을 채워준다.

In [10]:
# 뉴만 Kinesiology 3판
pre_90 = "Section I 운동학의 중요 주제 1\
Chapter 1 소개 · 3\
Chapter 2 인체 관절의 기본 구조와 기능 · 32\
Chapter 3 근육: 뼈대계통의 일차적인 안정자와 운동자 · 54\
Chapter 4 생체역학적 원리 · 88\
부록 I 삼각법 복습과 인체측정학 자료 · 130\
Section II 팔 133\
Chapter 5 어깨복합체 · 135\
Chapter 6 팔꿈치와 아래팔 · 195\
Chapter 7 손목 · 241\
Chapter 8 손 · 277\
부록 II 팔의 근육 부착부위와 신경지배, 근육의 단면적, 그리고 피부분절을 위한 참고 자료 · 337\
Section III 몸통뼈대 351\
Chapter 9 몸통뼈대: 골학 및 관절학 · 353\
Chapter 10 몸통뼈대: 근육과 관절의 상호작용 · 430\
Chapter 11 씹기와 환기의 운동학 · 479\
부록 III 말총 및 몸통뼈대 근육의 부착부위, 신경지배, 그리고 모멘트팔을 위한 참고 자료 · 514\
Section IV 다리 521\
Chapter 12 엉덩관절 · 523\
Chapter 13 무릎 · 588\
Chapter 14 발목과 발 · 650\
Chapter 15 걷기의 운동학 · 713\
Chapter 16 달리기의 운동학 · 773\
부록 IV 다리의 근육 부착부위와 신경지배, 근육의 단면적, 그리고 피부분절을 위한 참고 자료 · 799"

# 혈맥약침
pre_99 = "혈맥약침이 임상에 쓰이기 시작한 지 많은 시간이 지났지만, 아직도 여기저기 갑론을박이 많다. 학문에도 흥망성쇠가 있으니 새로운 기술과 융합하여 발전을 도모하는 것은 고무할 일이건만, 유독 한의계는 그러한 시도를 할 때마다 숱한 어려움을 겪어 왔다. \
그러나 생명을 다루는 의학이라면 발전을 위한 노력을 게을리하지 않는 것이 마땅한 의무이며, 고유의 본초 방제를 바탕으로 혈맥 투약이라는 방법을 융합한 혈맥약침은 한의학의 발전을 이끌 주요 분야의 하나가 될 것이라 믿는다. \
현재 국내 뿐 아니라 해외에서도 다양한 혈맥약침제제가 개발되어 임상에 활용되고 있으며, 혈맥약침에 관한 임상연구도 지속적으로 보고되고 있다. \
이 책은 임상에서 사용되는 다양한 혈맥약침제제와 이들을 활용한 임상연구보고들을 소개하고 있다. 아울러 독자들의 이해를 돕기 위해 혈맥약침의 이론적 기반이 되는 여러 기초 이론과 혈맥약침제제 및 혈맥약침시술에 관한 개괄적인 내용을 함께 수록하였다."

# 임상 한의사를 위한 기본 한약처방 강의
pre_144 = "임상 한의사를 위한 기본 한약처방 강의. 임상 초보를 위한 한의학사 간단 정리 2\
〈상한론〉의 역사 9\
허준이 〈동의보감〉을 완성하기 까지 15\
일본 의학사 간단 정리 17\
기본적인 한의학 용어의 역사에 관한 정리 20\
2. 역대 주요 서적 처방 해설\
〈상한론〉과 〈금궤요략〉의 類方들에 대한 개괄적인 이해 68\
〈상한론〉 병위(病位)에 따른 처방의 분류 88\
〈상한론〉〈금궤요략〉 처방의 후세로의 변화 97\
〈방약합편〉 유방 해설 107\
온병(溫病)에 관한 이야기 143\
상한과 온병 그리고 사상의학 149\
독감 그리고 온병 151\
감기 치료 시 구별해야 할 병증 154\
온병(溫病)의 주요 처방 해설 156\
〈귤창서영〉 정리 163\
〈의림개착〉 주요처방 해설 238\
〈의학충중참서록〉 주요처방 해설 242\
쉽게 이해하는 사상인의 병리(病理) 260\
〈동의수세보원〉 처방의 개괄적인 분석 264\
주요 약재와 처방들에 대한 기본적인 개념 정리 276\
3. 임상\
스트레스의 스테이지에 관하여 312\
스트레스에 대한 인체의 반응에 대응하는 약재 315\
한의학 기본 생병리에 관한 이야기 322\
수면에 대한 기본적인 이해 397\
수면장애의 치료 400\
불면증의 행동치료 405\
신경 정신 질환의 치료 처방 407\
식욕에 관한 기본적인 이해 422\
식욕을 안정적으로 유지하기 위한 생활습관 424\
식욕과 관련된 임상 이야기 426\
소화기 내과 초간단 정리 430\
소화기 환자 관리 시 유의사항 439\
급성장염에 관한 이야기 441\
소화기 질환의 치료 처방 443\
호흡에 관한 기본적인 이해 468\
호흡과 명상에 대하여 470\
호흡과 관련된 임상 이야기 473"

# 복증기람 복증기람익 세트
pre_168 = "본서는 『복증기람』만을 번역한 책이 아니다. 『복증기람』을 온전히 이해하려면 당대의 연관된 의서를 아울러 살펴봐야 한다. 『방극』『복증도휘』『도복증기람』『복증도해』를 완역하여 『복증기람』의 관련된 부분에 재배치함으로써 독자의 이해를 높이고자 하였다.\
(1) 총론 및 내경 진척(診尺) 도해 初編·上冊 _45\
(2) 중경의 복증 부위와 전신의 명칭, 삼양삼음·표리내외 도해 初編·上冊 _58\
(3) 신간동(腎間動)의 설과 도 初編·上冊 _65\
(4) 동계(動悸)의 변(辨) (附: 치법약안(治法略\案)) 初編·上冊 _70\
(5) 복중제괴(諸塊)의 변(辨)과 치(治) (부방9수(首), 도) 初編·上冊 _77\
(6) 복증진안법(腹證診按法)과 도 初編·下冊 _86\
Ⅰ. 『복증기람』 正編·上冊\
(0) 복증을 살피는 방법 _100\
(1) 계지탕의 도 _101\
讀계지탕의 증 _102\
圖解계지탕증의 도 _105\
圖解계지감초탕증 병인차수모심도 _109\
(2)-1 소시호탕의 도 _111\
圖彙소시호탕의 증 _113\
圖解소시호탕 흉협고만, 협하비경의 도 _114\
(2)-2 소시호탕의 도 (흉륵방창(胸肋肪脹)) _116\
圖彙소시호탕의 증 (흉륵방창) _118\
圖解흉협고만, 협륵팽창의 도 _119\
(3) 대시호탕의 도 _120\
圖彙대시호탕의 증 _121\
讀대시호탕의 증 _122\
圖解대시호탕의 도 _124\
(4) 시호가망초탕의 도 _126\
圖彙시호가망초탕의 증 _128\
讀시호가망초탕의 증 _129\
圖解시호가망초탕의 도 _130\
圖解시호가용골모려탕증의 도 _131\
(5) 오수유탕의 도 _132\
讀오수유탕의 증 _134\
(6) 인삼궁군도화탕의 도 _136\
讀인삼궁군도화탕의 증 _138\
(7)-1 도군원(도핵승기탕)의 1도 _139\
圖彙도핵승기탕의 증(1) _140\
(7)-2 도군원(도핵승기탕)의 2도 _141\
圖彙도핵승기탕의 증(2) _142\
讀도군원의 증 _143\
圖解도인승기탕증의 도 _145\
(8) 과루해백백주탕의 도 _147\
讀과루해백백주탕의 증 _149\
(9) 자석환의 도 _149\
讀자석환의 증 _150\
(10) 귤피대황박초탕의 도 _152\
讀귤피대황박초탕의 증 _153\
(11) 육군자탕의 도 _155\
圖解육군자탕 _15637\
(12) 대시호탕합감초건강탕의 도 _157\
讀대시호탕합감초건강탕의 증 _158\
(13) 대황감수탕의 도 _159\
圖彙대황감수탕의 증 _160\
讀대황감수탕의 증 _162\
圖解대황감수탕증의 도 _163\
(14)-1 대승기탕의 1도: 복견만 _165\
圖彙대승기탕의 증(1) _166\
圖解대승기탕 복견만증의 도 _167\
(14)-2 대승기탕의 2도: 독견만 _169\
圖彙대승기탕의 증(2) _170\
讀대승기탕의 증과 도 _171\
圖解대승기탕 조시의 도 _174\
(15) 소건중탕의 도 _177\
圖彙소건중탕의 증 _178\
讀소건중탕의 증 _179\
圖解소건중탕증의 도 _180\
(16) 계강조초황신부탕의 도 _182\
(附: 지출탕증과 견괴(堅塊)·결괴(結塊)·혈괴(血塊) 불해(不解)의 론)\
圖彙계강조초황신부탕의 증 _183\
讀계강조초황신부탕 _185\
圖解계지거작약가마황세신부자탕증의 도 (附: 지출탕의 변) _188"

# 외래진료에서 꼭 쓰이는 통증주사치료
pre_185 = "『통증 주사치료』는 전신에 걸쳐 다양한 질환의 발생 원인과 관련된 해부학, 초음파나 투시영상의 도움 없이 외래에서 쉽게 시행할 수 있는 주사 방법을 소개하고 있습니다. 각 질환의 병리학적 발병 원인, 주사 치료를 할 경우 주의 해야 하는 구조물, 발생 가능한 합병증, 대표적인 신체 검진 방법, 영상 검사 소견뿐만 아니라 사체 사진과 실제 수술 사진 등 다양한 도해로 이해를 도왔다."

# 약의 의문
pre_187 = "이 책에는 복제의약품, 약물 부작용, 의사와 환자 관계, 한약의 사용에 관한 것은 물론 잘 취급되지 않지만 상당히 중요하고 재미있는 주제도 포함되어 있다. 기본적으로 질문에 답하는 형식이라 더욱 흥미있을 것이다. 질문의 내용은 의사가 진료실에서 매일 만나고 고민하는 문제의 많은 부분을 포함하고 있어, 실제적으로 유익하다. 내용은 내과 중심이지만 의학 전반에 걸쳐 다루고 있어, 매일 거의 전 분야의 환자를 진료하는 개업의 선생님과 다른 과 진료내용이 필요한 전문의 선생님에게도 좋은 참고자료가 될 것 이다.\
Q1 항생제가 필요한 감기의 형태는?\
Q2 주사제에서 경구용으로 변경하는 것을 포함한 투여중인 항생제의 투여 지속 기간은?\
Q3 상처 부위의 세균감염 예방에 항생제 효과는?\
Q4 해외에서 국내보다 높은 용량의 투여가 권장되는 경우, 항생제 투여량은?\
Q5 비슷한 항균스펙트럼을 가진 항생제의 선택 방법은?\
Q6 항생제 투여 중 세균배양을 위해 검체를 채취할 경우 실시하기 적당한 시기는?\
Q7 항생제 투여용량을 결정하는 경우 기준이 되는 GFR에 적당한 측정법 공식은?\
Q8 수술 후 항생제 투여 중 발열이 있는 경우에 고려되는 원인은?\
Q9 항생제가 효과 없다고 판단되는 상황은?\
Q10 임상증상이 호전되었는데도 WBC, CRP 등의 검사치가 높은 경우 45 항생제의 중지 여부는?\
2. 스테로이드\
Q11 스테로이드 외용제는 어느 정도 사용 시 부작용이 생기나?\
Q12 급성 호흡곤란증후군(ARDS) 환자에서 스테로이드 치료 효과는?\
Q13 혈당조절이 잘 되지 않는 당뇨병환자가 천식발작이 있을 때, 52 스테로이드의 전신투여는?\
3. 해열진통제"

pre_233 = '는 총 15파트로 구성하여, 총론부터 신경과 근육에서의 막생리, 체액과 콩팥, 신경계, 내분비와 생식,스포츠 등을 다룬 책이다.'

pre_293 = '실전 한약가이드에 대해 다룬 도서입니다 기초적이고 전반적인 내용을 확인할 수 있도록 구성했습니다 한의학의 역사와 기본 개념   동양의학의 유입과 한의학의 변천   한의학의 기초개념  한의학 진단과 치료   진단에 대해   치료에 대해   동양의학과 서양의학의 상보적 결합 동서의학 결합을 향해   낡고 새로운 한의학  동서의학 결합을 향해   의학 예방의학으로서 한의학    전후 일본 한의학계의 발자취를 알기에는   음양과 허실의 차이   오장의 개념   증 파악은 어렵다    소아는 양증 고령자는 허증이 많다   과 임상의학   한약재란   한약 이름 유래   증상별 한약 사용법    전신 증상    발열성인 발열소아 발진성인 발진소아 부종성인 부종소아 냉증 화끈거림 피로권태감 침한 도한 상기  신경계 증상    두통 어지럼증 경련 어깨 결림   순환기계 증상    동계 숨가쁨 흉통  호흡기계 증상    기침 천명과 호흡곤란   소화기계 증상      오심 구토 상복부 부정수소 식욕부진 복통 설사 변비 복부팽만   질환별 한약 사용법   중추신경계     자율신경 실조증 우울증 신경증공황 장애 등불면증인지증간질편두통 순환기    본태성 고혈압 본태성 저혈압 기립성 저혈압 허혈성 심질환관상동맥질환만성심부전 부정맥   호흡기    감기증후군 인플루엔자 기관지천식 만성기관지염 만성폐쇄성폐질환  소화기 역류성 식도염 만성위염 기능성 소화불량  소화성궤양 과민성장증후군 궤양성 대장염 크론 병 만성간염 간경변 담낭염 담관염 담석증 담낭 운동이상 만성 췌장염딸꾹질  혈액내과     빈혈 특발 혈소판 감소 자색반병  자가면역   교원병 전신 홍반 루푸스 쇼그렌 증후군  대사 내분비 당뇨병 비만증 지질이상증 마름여윔 갑상선기능이상통풍  소아과 허약체질 기립성 조절장애 바이러스 감염증 야뇨증 야제 야경증 신비뇨기 만성 신질환사구체신염 신증 요로감염증만성 방광염을 포함 요로결석증 전립선비대증 남성 갱년기장애 남성 난임 요로부정수소  외과 장폐색 치 질환 암 수술 후 체력 면역력 저하 항암치료에 따르는 부작용  부인과   월경이상월경불순 월경곤란증과 월경전증후군 난임 불육증 습관성 유산 임신오조 절박유산 조산 임신고혈압증후군 임신중 감기 변비 빈혈 산욕기의 이상 갱년기장애  정형외과 관절 류마티스 요통 좌골신경통 변형성 무릎관절증 어깨관절 주위염 사지 관절통 근육 경련  피부과 아토피성 피부염 만성습진 두드러기 주사 원형탈모증 피부소양증 동창동상 대상포진 이비인후과 알레르기성 비염 만성부비동염 편도염 메니에르 병 인후두이상감각 중이염 난청감음난청 비출혈 이명 쉰 목소리  안과  안구건조증 안정피로 백내장 녹내장 안저출혈  치과 구강외과 구내염 설염 혀 통증 치주병 구강건조증 턱관절증 발치 후 동통 비정형 치통   방제편 한약 투여방법과 주의사항  방제군 별로 본 한약    마황제  마황탕 월비가출탕 마행감석탕 오호탕 마황부자세신탕 마행의감탕 의이인탕 갈근탕 갈근탕가천궁신이 소청룡탕 신비탕  계지탕류  계지탕 계지가출부탕 계지가용골모려탕 계지가작약탕 계지가작약대황탕 당귀사역가오수유생강탕 당귀건중탕 소건중탕 황기건중탕  시호제와 관련 방제  소시호탕 소시호탕가길경석고 대시호탕 시호가용골모려탕 사역산 시박탕 시령탕 시호계지탕 시호계지건강탕 시함탕  시호를 포함한 방제시호제 제외  십미패독탕 시호청간탕 신비탕 을자탕 가미소요산 자음지보탕 형개연교탕 억간산 억간산가진피반하 보중익기탕 죽여온담탕 가미귀비탕  인삼탕류와 삼기제   인삼탕류  인삼탕 계지인삼탕 복령음 대건중탕 사군자탕 육군자탕 계비탕  삼기제  반하백출천마탕 보중익기탕 청심연자음 청서익기탕 십전대보탕 귀비탕  지황제   사물탕류보혈제  칠물강하탕 사물탕 궁귀교애탕 자음강화탕 당귀음자 십전대보탕 인삼양영탕  팔미지황환류보신제  팔미지황환 우차신기환 육미환  부자제  진무탕 대방풍탕 계지가출부탕 마황부자세신탕 팔미지황환 우차신기환  사심탕류와 관련방제  삼황사심탕 황련해독탕 온청음 반하사심탕 황련탕 시함탕 여신산 청상방풍탕 시호청간탕 형개연교탕  승기탕류와 관련방제  대승기탕 조위승기탕 윤장탕 대황감초탕 마자인환 도핵승기탕 방풍통성산  맥문동을 포함한 방제  조등산신이청폐탕 맥문동탕 청심연자음 자음강화탕 자감초탕 죽여온담탕 자음지보탕 청서익기탕 온경탕 청폐탕  후박향부자소엽을포함한 주요한 방제  시박탕 이출탕 반하후박탕 평위산 삼소음 천궁다조산 오적산 여신산 당귀탕 향소산  대황제   반하제   석고제   청열제  삼물황금탕 백호가인삼탕 용담사간탕 오림산 치두창일방 인진호탕 소풍산 신이청폐탕 청폐탕 온청음 배농산급탕  기제   구어혈제  도핵승기탕 대황목단피탕 통도산 계지복령환 계지복령환가의이인 치타박일방당귀작약산 여신산 가미소요산 온경탕  이수제   전신형  오령산 인진오령산 저령탕 저령탕합사물탕 영계출감탕 영강출감탕 진무탕  심하형  위령탕 소반하가복령탕 이진탕 육군자탕 복령음 복령음합반하후박탕인삼탕  관절형 흉부형  이출탕 방기황기탕 목방기탕 소청룡탕 영감강미신하인탕  주로 돈복증상이 나타났을 때 복용하는 약  작약감초탕 입효산 길경탕 소반하가복령탕  기타 방제  승마갈근탕 산조인탕 감맥대조탕 소경활혈탕 오수유탕 안중산 주요 한약재 마황 계지 시호 황금 인삼 황기 지황 부자 황련 맥문동 후박 향부자 소엽 대황 반하 석고 지모 도인 목단피 작약 당귀 복령 택사 창출  참고백출 '

### 2. 1만 글자 이하로 전처리(1만 글자가 넘는 경우 api가 처리하지 못함)

In [11]:
# 1만 글자 넘는 경우 확인
# for i in range(len(df)):
#     if len(df['description'].iloc[i]) >= 10000:
#         print(df.iloc[i])
        
# 1만 글자 넘는 것은 띄어쓰기 없애는 전처리 진행
df['description'][df.index == 126] = df[df.index == 126].description.values[0].replace(" ",'')
df['description'][df.index == 229] = df[df.index == 229].description.values[0].replace(" ",'')    

In [12]:
# description의 내용이 '  '라서 구글링해서 내용 넣음
df['description'][df.index == 90] = pre_90 
df['description'][df.index == 99] = pre_99
df['description'][df.index == 144] = pre_144
df['description'][df.index == 168] = pre_168
df['description'][df.index == 185] = pre_185
df['description'][df.index == 187] = pre_187
df['description'][df.index == 233] = pre_233
df['description'][df.index == 293] = pre_293

# name_x + description 내용을 담은 column
df["full_content"] = df['name_x'] + ' ' + df['description']

#  한글을 제외한 글자를 제거
import re    
df['full_content'] = df['full_content'].apply(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x))

In [13]:
df.head()

,_id,category_id_x,date_created,name_x,description,meta_description_x,meta_title,tags_x,three_months,regular_price_y,m_name,L_name,s_name,book_corpus,book_corpus_konlp,full_content
0,5d13115e32026c0b35383897,5cf8bbba0098b2225c5dfaa3,{'$date': '2019-06-26T06:31:58.090Z'},KCD 한방내과 진찰진단 가이드라인,의 번째 큐레이션 상시판매 권승원의 한방내과 진찰진단 가이드라인 의학은빠른속...,진료에 어려움 있을 때 먼저 찾는 책,KCD 한방내과 진찰진단 가이드라인,[가온해미디어],35,150000.0,전문,도서,[가온해미디어],"KCD 한방내과 진찰진단 가이드라인, 의 번째 큐레이션 상시판매 권승원의 한방내과...","[정리, 보험, 션, 현장, 년, 참고, 신부전, 다작, 위해, 대한, 제시, 의,...",한방내과 진찰진단 가이드라인 의 번째 큐레이션 상시판매 권승원의 한방내과 진찰...
1,5d9edfe70dabe405b156e213,5cf8bbba0098b2225c5dfaa3,{'$date': '2019-10-10T07:38:15.467Z'},우리 아이 감기,우리 아이 감기부제 약물 남용과 과잉치료가 없는 새로운 감기 진료를 목표로 ...,약물남용과 과잉치료가 없는 새로운 감기 진료를 목표로,우리 아이 감기,[물고기숲],7,NaN,None,None,None,"우리 아이 감기,우리 아이 감기부제 약물 남용과 과잉치료가 없는 새로운 감기 진료를...","[기적, 소아과학, 왜곡, 위해, 초음파, 가장, 모두, 삼, 보호자, 중증, 폐렴...",우리 아이 감기 우리 아이 감기부제 약물 남용과 과잉치료가 없는 새로운 감기 진료를...
2,5e3cb2904267e105dfff13b3,5cf8bbba0098b2225c5dfaa3,{'$date': '2020-02-07T00:42:56.629Z'},통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,_ 출판사 리뷰년전 소수 인원이 모여 시작한 통증 공부 모임이 작은 불씨가 ...,하이드로다이섹션: 포도당을 이용한 신경의 유착박리와 신경주변 환경개선,통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,[메디안북],2,190000.0,전문,도서,[메디안북],"통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉, _ 출판사 리뷰년전 소수...","[골반, 불씨, 부, 모임, 현장, 유착, 밸런스, 좌, 관절, 척수, 배늑, 위해...",통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉 출판사 리뷰년전 소수 ...
3,5e3d06b64267e105dfff13e0,5cf8bbba0098b2225c5dfaa3,{'$date': '2020-02-07T06:41:58.622Z'},의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),의사들이 꼭 알고 있어야 하는 응급의학 실전지침서,,의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),[메디안북],0,130000.0,전문,도서,[메디안북],"의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),의사들이 꼭 알고 있어야...","[실전, 알, 의사, 꼭, 지침, 응급의학]",의사들이 꼭 알고 있어야 하는 응급의학 실전지침서 의사들이 꼭 알고 있어야 하는 응...
4,5e3d0da94267e105dfff13ed,5cf8bbba0098b2225c5dfaa3,{'$date': '2020-02-07T07:11:37.566Z'},복부 초음파 정석,_ 저자서문 초음파 검사는 임상 영역에서 청진기로 불릴 정도로 간편한...,정확한 복부초음파를 위한 필독서,복부 초음파 정석,[메디안북],1,60000.0,전문,도서,[메디안북],"복부 초음파 정석, _ 저자서문 초음파 검사는 임상 영역에서 청진기로 ...","[주, 초음파, 성능, 장단점, 알코올, 한눈, 만성, 모두, 직경, 김현정, 효율...",복부 초음파 정석 저자서문 초음파 검사는 임상 영역에서 청진기로 불...


# corpus에 형태소 분석 결과 담기

In [14]:
from tqdm import tqdm

corpus = []
for i in tqdm(range(len(df))):
    con = df['full_content'].iloc[i]
    # 형태소 분석
    data = etri_sentence_analysis.get_analyzed_sentence(con, 'morp')
    corpus.append(data)
len(corpus)

100%|██████████| 372/372 [02:31<00:00,  2.46it/s]


372

# dataframe

In [15]:
# dataframe 초기화
new_df = pd.DataFrame(df[['_id','name_x','full_content']])
new_df['tokens'] = None

# dataframe에 각 토큰 담기
for i in tqdm(range(len(corpus))):
    nouns = []
    res3 = corpus[i]['sentence'][0]['morp'] # 형태소 결과
    
    for j in res3:
        #  형태소가 명사이고 2글자 이상인 것만 담기
        if j['type'] == 'NNG' and len(j['lemma']) > 1: 
            nouns.append(j['lemma'])
            new_df['tokens'].iloc[i] = nouns

100%|██████████| 372/372 [00:13<00:00, 26.64it/s]


In [16]:
# 중복되는 토큰제거
new_df['tokens'] = new_df['tokens'].apply(lambda x: list(set(x)))

In [17]:
new_df.head()

,_id,name_x,full_content,tokens
0,5d13115e32026c0b35383897,KCD 한방내과 진찰진단 가이드라인,한방내과 진찰진단 가이드라인 의 번째 큐레이션 상시판매 권승원의 한방내과 진찰...,"[하나, 최근, 은빠, 라인, 선배, 전꼭, 영상, 한방, 필요, 참고, 배치, 구..."
1,5d9edfe70dabe405b156e213,우리 아이 감기,우리 아이 감기 우리 아이 감기부제 약물 남용과 과잉치료가 없는 새로운 감기 진료를...,"[교과, 약물, 제목, 양방, 중요, 안녕, 필요, 소아, 생각, 의원, 목표, 빈..."
2,5e3cb2904267e105dfff13b3,통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉 출판사 리뷰년전 소수 ...,"[걸음, 기존, 공부, 음파, 동안, 주제, 회복, 영역, 리뷰년, 선생, 하이드로..."
3,5e3d06b64267e105dfff13e0,의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),의사들이 꼭 알고 있어야 하는 응급의학 실전지침서 의사들이 꼭 알고 있어야 하는 응...,"[의사, 의학, 지침, 응급, 실전]"
4,5e3d0da94267e105dfff13ed,복부 초음파 정석,복부 초음파 정석 저자서문 초음파 검사는 임상 영역에서 청진기로 불...,"[부족, 음파, 고분, 차이, 정석, 저자, 영역, 검진, 핵심, 객관, 영상, 필..."


# save to json

In [22]:
# new_df.to_json('/home/user_4/CBF/Token/final_tokens_book.json')